In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx 
import os
from scipy.stats import spearmanr,pearsonr
import sys
from math import log10
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from csv import writer 
from sklearn import preprocessing
from scipy import stats

In [28]:
NPX_data =  pd.read_csv('4manuscript/cleaned_NPX_data.csv')
NPX_data = NPX_data.dropna(axis=1)

In [29]:
patient_data = pd.read_csv('4manuscript/cleaned_patient_data.csv')

In [30]:
target = ['sis3'] #['facit']    change targets to facit or sis3 based on the interests
add_fea = ['timesince','age']#,'size','gender','nihss',]  # change the features list to investigate their predictive powers
str_add_fea = 'pr +' + '+ '.join(add_fea)
print(str_add_fea)
y_data_features = []
y_data_features.extend(add_fea)
y_data_features.extend(target)
y_data_features.extend(['pid'])
y_data = patient_data.dropna(subset=target)
y_data = y_data[y_data_features] #'size','timesince','age',
y_data = y_data.dropna(subset=add_fea)

X = NPX_data

data = y_data.merge(X,how='inner', on='pid')

selected_features = [col for col in NPX_data if 'OID' in col]
selected_features.extend(add_fea)
pids = set(data['pid'].values.tolist())   

pr +timesince+ age
1011


In [31]:
def fs_func(fs_method, fs_x_tr, fs_y_tr, cv_n):
    X = fs_x_tr.values
    y = fs_y_tr.values
    if fs_method == 'high_sig':
        feat1 = get_highly_sig_prs()
        feat2 = add_fea
        k_feas = []
        k_feas.extend(feat1)
        k_feas.extend(feat2)
        return k_feas
    if fs_method == 'RF':
        model = LogisticRegression()
        rfe = RFE(model, 100)
        fit = rfe.fit(X, y)
        lst = zip(fit.ranking_, fs_x_tr.columns)
        lst = sorted(lst, key = lambda x:np.abs(x[0]))
    elif fs_method == 'l2':
        scaler = StandardScaler()
        scaler.fit(fs_x_tr)
        sel_ = SelectFromModel(LogisticRegression(penalty='l2'))
        sel_.fit(scaler.transform(fs_x_tr), fs_y_tr)
        ridge_selected_feat = fs_x_tr.columns[(sel_.get_support())]
        lst = zip(sel_.estimator_.coef_[0,:], fs_x_tr.columns)
        print(sel_.estimator_.coef_)
        lst = sorted(lst,  key = lambda x:np.abs(x[0]))
        ridge_df = pd.DataFrame(lst, columns =['feature','Score'])
        ridge_df ['cv']=cv_n
        write_to_file(fs_method, ridge_df)
        return ridge_selected_feat
    elif fs_method == 'MIR':
        test = SelectKBest(score_func=mutual_info_regression, k=100)
        fit = test.fit(X, y)
        lst = zip(fit.scores_, fs_x_tr.columns)
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    else:
        return fs_x_tr.columns
    fea_df = pd.DataFrame(lst, columns =['feature','Score'])
    fea_df ['cv']=cv_n
    write_to_file(fs_method, fea_df)
    feas = [name for coef, name in lst[:100]]
    #print(lst)
    return feas

In [32]:
model = LogisticRegression(fit_intercept=True)
tru_vals = []
predictions = []
fs_method = 'no filter' #'high_sig'# change no_filter to 'high_sig' to select highly significant proteins 
for i,p in enumerate(list(pids)):
    fs_x_tr =  data[data['pid']!=p][selected_features]
    fs_y_tr = data[data['pid']!=p][target]
    if fs_method == 'no filter':
        new_selected_features = selected_features
    else:
        new_selected_features = fs_func(fs_method, fs_x_tr, fs_y_tr, p)
    x_tr = data[data['pid']!=p][new_selected_features].values.reshape(-1, len(new_selected_features))
    y_tr = data[data['pid']!=p][target].values.reshape(-1, 1)
    x_te = data[data['pid']==p][new_selected_features].values.reshape(-1, len(new_selected_features))
    y_te = data[data['pid']==p][target].values.reshape(-1, 1)
    tru_vals.extend(y_te.flatten())
    model.fit(x_tr, y_tr)
    y_pred = model.predict(x_te)
    predictions.extend(y_pred)
List = [type(model).__name__, fs_method+str_add_fea, target,  spearmanr(predictions, tru_vals)[0], spearmanr(predictions, tru_vals)[1],  -log10(spearmanr(predictions, tru_vals)[1])]

In [33]:
def get_highly_sig_prs():
    df = pd.read_csv('4manuscript/%s_highly_corr_prs.csv'%target[0])
    print(df['OID'].values.tolist())
    return df['OID'].values.tolist()